In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv(r"C:\Users\palas\Downloads\surat_flat\surat_uncleaned.csv")

# ✅ 1) Drop broken/misaligned rows
# Rule: floor usually contains "out of"
bad_rows = df[~df["floor"].astype(str).str.contains("out of", na=False)]
df = df.drop(bad_rows.index)

# ✅ 2) Convert square_feet: "644 sqft" -> 644
df["square_feet"] = df["square_feet"].astype(str).str.replace("sqft","", regex=False).str.strip()
df["square_feet"] = pd.to_numeric(df["square_feet"], errors="coerce")

# ✅ 3) Convert price_per_sqft: "₹2,891 per sqft" -> 2891
df["price_per_sqft"] = df["price_per_sqft"].astype(str)
df["price_per_sqft"] = df["price_per_sqft"].str.replace("₹","", regex=False)
df["price_per_sqft"] = df["price_per_sqft"].str.replace(",","", regex=False)
df["price_per_sqft"] = df["price_per_sqft"].str.replace("per sqft","", regex=False).str.strip()
df["price_per_sqft"] = pd.to_numeric(df["price_per_sqft"], errors="coerce")

# ✅ 4) Convert price: "₹33.8 Lac" -> 3380000
def convert_price(x):
    if pd.isna(x):
        return np.nan
    x = str(x).lower().replace("₹","").replace(",","").strip()
    num = re.findall(r"[\d.]+", x)
    if not num:
        return np.nan
    num = float(num[0])
    if "lac" in x or "lakh" in x:
        return num * 100000
    if "cr" in x or "crore" in x:
        return num * 10000000
    return num

df["price"] = df["price"].apply(convert_price)

# ✅ 5) Drop missing price (target)
df = df.dropna(subset=["price"])

print("✅ Cleaned shape:", df.shape)
df.head()


✅ Cleaned shape: (3490, 11)


,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
0,2 BHK Apartment for Sale in Dindoli Surat,Carpet Area,644.0,New Property,Poss. by Oct '24,5 out of 10,Unfurnished,West,"Luxury project with basement parking, Solar ro...",2891.0,3380000.0
1,2 BHK Apartment for Sale in Althan Surat,Super Area,1278.0,New Property,Poss. by Jan '26,6 out of 14,Unfurnished,South -West,2 And 3 BHK Luxurious Flat for Sell In New Alt...,3551.0,4540000.0
2,2 BHK Apartment for Sale in Pal Gam Surat,Super Area,1173.0,Resale,Ready to Move,5 out of 13,Semi-Furnished,East,This affordable 2 BHK flat is situated along a...,3800.0,4460000.0
3,2 BHK Apartment for Sale in Jahangirabad Surat,Carpet Area,700.0,New Property,Ready to Move,6 out of 14,Unfurnished,East,2 BHK Flat For sell IN Jahangirabad Prime Loca...,3966.0,4700000.0
7,Office Space for Sale in Vesu Surat,Carpet Area,700.0,New Property,Poss. by Sep '25,7 out of 10,1,NaN,"best commercial for business ,investors ,busin...",3392.0,4410000.0


In [2]:
df["current_floor"] = df["floor"].str.extract(r"(\d+)").astype(float)
df["total_floor"] = df["floor"].str.extract(r"out of\s*(\d+)").astype(float)


In [3]:
df["furnishing"] = df["furnishing"].astype(str)

df.loc[~df["furnishing"].isin(["Furnished","Semi-Furnished","Unfurnished"]), "furnishing"] = "Unknown"


In [4]:
df["facing"] = df["facing"].fillna("Unknown")


In [5]:
df["bhk"] = df["property_name"].str.extract(r"(\d+)\s*BHK")
df["bhk"] = df["bhk"].astype(float)
df["bhk"] = df["bhk"].fillna(df["bhk"].median())


In [6]:
df.drop(columns=["description"], inplace=True)


In [7]:
df = df[~df["property_name"].str.contains("Office|Shop|Showroom|Commercial", case=False, na=False)]


In [8]:
df["location"] = df["property_name"].str.extract(r"in\s+(.*)\s+Surat", expand=False)
df["location"] = df["location"].fillna("Unknown")


In [9]:
# price outlier remove
Q1 = df["price"].quantile(0.25)
Q3 = df["price"].quantile(0.75)
IQR = Q3 - Q1
df = df[(df["price"] >= Q1 - 1.5*IQR) & (df["price"] <= Q3 + 1.5*IQR)]

# sqft outlier remove
Q1 = df["square_feet"].quantile(0.25)
Q3 = df["square_feet"].quantile(0.75)
IQR = Q3 - Q1
df = df[(df["square_feet"] >= Q1 - 1.5*IQR) & (df["square_feet"] <= Q3 + 1.5*IQR)]


In [10]:
df_ml = pd.get_dummies(df, drop_first=True)
print(df_ml.shape)


(2714, 2317)


In [11]:
df_ml = df_ml.replace([np.inf, -np.inf], np.nan)
df_ml = df_ml.fillna(df_ml.median(numeric_only=True))


In [12]:
df_ml.to_csv("surat_flat_ml_ready.csv", index=False)
print("✅ Saved: surat_flat_ml_ready.csv")


✅ Saved: surat_flat_ml_ready.csv


In [13]:
from sklearn.model_selection import train_test_split

X = df_ml.drop("price", axis=1)
y = df_ml["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [16]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)

rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)

print("✅ Random Forest Results")
print("MAE :", mean_absolute_error(y_test, pred_rf))
print("RMSE:", np.sqrt(mean_squared_error(y_test, pred_rf)))
print("R2  :", r2_score(y_test, pred_rf))


✅ Random Forest Results
MAE : 550575.0153468386
RMSE: 1330043.278932162
R2  : 0.9343208024014152


In [17]:
import joblib

joblib.dump(rf, "surat_price_model.pkl")
joblib.dump(X_train.columns, "surat_columns.pkl")

print("✅ Saved: surat_price_model.pkl")
print("✅ Saved: surat_columns.pkl")


✅ Saved: surat_price_model.pkl
✅ Saved: surat_columns.pkl


In [18]:
df_ml.to_csv("surat_flat_ml_ready.csv", index=False)
print("✅ Saved: surat_flat_ml_ready.csv")


✅ Saved: surat_flat_ml_ready.csv
